In [57]:
import torch

In [1]:
from dataset.dataset import ML1MDataset
from dataloader.dataloader import BertDataloader, BertEvalDataset

/Users/kyeongchanlee/.pyenv/versions/3.9.9/envs/candidate/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from options import args
dataset = ML1MDataset(args)

In [3]:
dataset_path = dataset._get_preprocessed_dataset_path()
if not dataset_path.parent.is_dir():
    dataset_path.parent.mkdir(parents=True)

In [4]:
df = dataset.load_ratings_df()

raw_code's return : ml-1m


/Users/kyeongchanlee/PycharmProjects/BertCustom/BertCustom/dataset/dataset.py:61: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep='::', header=None)


In [5]:
df = dataset.make_implicit(df)

Turning into implicit ratings


In [6]:
df = dataset.filter_triplets(df)

Filtering triplets


In [7]:
df, u_i_map = dataset.densify_index(df, target_cols=['sid', 'uid'])

Densifying index


In [8]:
df

,uid,sid,rating,timestamp
0,1,1105,5,978300760
1,1,640,3,978302109
2,1,854,3,978301968
3,1,3178,4,978300275
4,1,2163,5,978824291
...,...,...,...,...
1000204,6040,1020,1,956716541
1000205,6040,1023,5,956704887
1000206,6040,549,5,956704746
1000207,6040,1025,4,956715648


In [9]:
min(df.uid)

1

In [10]:
min(df.sid)

1

In [11]:
def split_tra_val_tes(df, user_count):
    if args.split == 'leave_one_out':
        print('Splitting')
        user_group = df.groupby('uid')
        user2items = user_group.progress_apply(lambda u_df: list(u_df.sort_values(by='timestamp')['sid']))
        # user2genres = user_group.progress_apply(lambda d: list(d.sort_values(by='timestamp')['gid']))

        train, val, test = {}, {}, {}

        # train_g, val_g, test_g = {}, {}, {}
        for user in range(1, user_count+1):
            items = user2items[user]
            train[user], val[user], test[user] = items[:-2], items[-2:-1], items[-1:]

            # genres = user2genres[user]
            # train_g[user], val_g[user], test_g[user] = genres[:-2], genres[-2:-1], genres[-1:]

        return train, val, test

In [12]:
tra_item_seq, val_item_seq, tes_item_seq = split_tra_val_tes(df, len(u_i_map['uid']))

Splitting


100%|██████████| 6040/6040 [00:01<00:00, 3655.47it/s]


In [13]:
def load_attrs_df(self):
    '''
    Return : 
        attr_df = itemid |   genre  |  ...
                    1    | 'comedy' |
    '''
    attr_df = dataset.load_movies_df()
    # attr_df = attr_df[attr_df['sid'].isin(rating_df['sid'])]
    return attr_df[['sid'] + dataset.attributes]

In [14]:
attrs_df = load_attrs_df(df)

/Users/kyeongchanlee/PycharmProjects/BertCustom/BertCustom/dataset/dataset.py:68: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep='::', header=None, encoding='ISO-8859-1')


raw_code's return : ml-1m


In [15]:
attrs_df['sid'] = attrs_df['sid'].map(u_i_map['sid'])

In [16]:
attrs_df

,sid,genre
0,1.0,Animation|Children's|Comedy
1,2.0,Adventure|Children's|Fantasy
2,3.0,Comedy|Romance
3,4.0,Comedy|Drama
4,5.0,Comedy
...,...,...
3878,3702.0,Comedy
3879,3703.0,Drama
3880,3704.0,Drama
3881,3705.0,Drama


In [17]:
attrs_df, amap = dataset.densify_index(attrs_df, target_cols=dataset.attributes)

Densifying index


In [18]:
attrs_df

,sid,genre
0,1.0,179
1,2.0,239
2,3.0,33
3,4.0,150
4,5.0,160
...,...,...
3878,3702.0,160
3879,3703.0,254
3880,3704.0,254
3881,3705.0,254


In [19]:
i2attr_map = []
for attr_name in dataset.attributes:
    i2attr_map.append(dict(zip(attrs_df['sid'], attrs_df[attr_name])))

In [20]:
if not dataset_path.parent.is_dir():
    dataset_path.parent.mkdir(parents=True)

In [21]:
dataset_path = dataset._get_preprocessed_dataset_path()
dataset_path

PosixPath('Data/preprocessed/min_rating0-min_uc5-min_sc0-splitleave_one_out/dataset.pkl')

In [22]:
import pickle
dataset_pickle = {'tra_item_seq': tra_item_seq,  # dict : {densified user's id : [3, 4, ...](densified item's id)}
            'val_item_seq': val_item_seq,
            'tes_item_seq': tes_item_seq,
            'u_i_map' : u_i_map,
            'i2attr_map' : i2attr_map,
            'amap' : amap 
            }
with dataset_path.open('wb') as f:
    pickle.dump(dataset_pickle, f)

# dataloader

In [23]:
dataloader = BertDataloader(args, dataset)

Already preprocessed. Skip preprocessing
Negatives samples exist. Loading.
Negatives samples exist. Loading.


In [24]:
# train_loader, val_loader, test_loader = dataloader.get_pytorch_dataloaders()


In [25]:
prep_data = dataset.load_dataset()

Already preprocessed. Skip preprocessing


In [26]:
import random
seed = args.dataloader_random_seed
rng = random.Random(seed)
train = prep_data['tra_item_seq']
val = prep_data['val_item_seq']
test = prep_data['tes_item_seq']

umap = prep_data['u_i_map']['uid']
smap = prep_data['u_i_map']['sid']

In [27]:
user_count = len(umap)
item_count = len(smap)


In [28]:
args.num_items = len(smap)

In [29]:
args.num_items

3706

In [30]:
max_len = args.bert_max_len
mask_prob = args.bert_mask_prob
CLOZE_MASK_TOKEN = item_count + 1

In [31]:
max_len

100

In [32]:
i2attr_map = prep_data['i2attr_map']

In [33]:
use_attributes = args.use_attributes

In [34]:
attrs_each_size = [len(_map) for _map in amap.values()]
args.attrs_each_size = attrs_each_size

In [35]:
attrs_each_size

[301]

In [36]:
ATTRS_MASK_TOKENS = [size+1 for size in attrs_each_size]
ATTRS_MASK_TOKENS

[302]

In [37]:
for _map, m_token in zip(i2attr_map, ATTRS_MASK_TOKENS):
    _map[CLOZE_MASK_TOKEN] = m_token  # item's mask token to attr's mask token
    _map[0] = 0  # pad token too

In [38]:
i2attr_map[0][3707]

302

In [39]:
from dataloader.negative_samplers import negative_sampler_factory
save_folder = dataset._get_preprocessed_folder_path()

code = args.train_negative_sampler_code
train_negative_sampler = negative_sampler_factory(code, train, val, test,
                                                    user_count, item_count,
                                                    args.train_negative_sample_size,
                                                    args.train_negative_sampling_seed,
                                                    save_folder)
code = args.test_negative_sampler_code
test_negative_sampler = negative_sampler_factory(code, train, val, test,
                                                    user_count, item_count,
                                                    args.test_negative_sample_size,
                                                    args.test_negative_sampling_seed,
                                                    save_folder)

train_negative_samples = train_negative_sampler.get_negative_samples()
test_negative_samples = test_negative_sampler.get_negative_samples()

Negatives samples exist. Loading.
Negatives samples exist. Loading.


# train_dataset

In [42]:
import torch.utils.data as data_utils
class BertTrainDataset(data_utils.Dataset):
    def __init__(self, u2seq, max_len, mask_prob, seq_mask_token, num_items, rng, i2attr_map, use_attributes):
        self.u2seq = u2seq
        self.users = sorted(self.u2seq.keys())
        self.max_len = max_len
        self.mask_prob = mask_prob
        self.seq_mask_token = seq_mask_token
        self.num_items = num_items
        self.rng = rng

        self.use_attributes = use_attributes
        self.i2attr_map = i2attr_map

        self.printer = PrintInputShape(3)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        user = self.users[index]
        seq = self.u2seq[user]

        return user, seq

        # tokens = []
        # labels = []
        # attrs = []

        # for s in zip(seq):
        #     prob = self.rng.random()
        #     if prob < self.mask_prob:
        #         prob /= self.mask_prob

        #         if prob < 0.8:
        #             tokens.append(self.seq_mask_token)
        #         elif prob < 0.9:
        #             tokens.append(self.rng.randint(1, self.num_items))
        #         else:
        #             tokens.append(s)

        #         labels.append(s)
        #     else:
        #         tokens.append(s)
        #         labels.append(0)

        # tokens = tokens[-self.max_len:]
        # labels = labels[-self.max_len:]

        # mask_len = self.max_len - len(tokens)

        # tokens = [0] * mask_len + tokens
        # labels = [0] * mask_len + labels

        # # print(tokens)
        # if self.use_attributes:
        #     for _map in i2attr_map:
        #         attr_idx = torch.LongTensor([_map[item] for item in tokens])
        #         attrs.append(attr_idx)

        # return torch.LongTensor(tokens), torch.LongTensor(attrs), torch.LongTensor(labels)

from utils import PrintInputShape

In [43]:
class MyTrainCollator:
    def __init__(self, rng, seq_mask_token, num_items, max_len, use_attributes, i2attr_map):
        self.rng = rng
        self.seq_mask_token = seq_mask_token
        self.num_items = num_items
        self.max_len = max_len
        self.use_attributes = use_attributes
        self.i2attr_map = i2attr_map

    def __call__(self, batch):
        
        # users_batch = [_[0] for _ in batch]
        seqs_batch = [_[1] for _ in batch]

        seqs_output = []
        labels_output = []
        attrs_output = []

        for seq in seqs_batch:
            one_seq = []
            one_seq_labels = []
            for item in seq:
                prob = self.rng.random()
                if prob < mask_prob:
                    prob /= mask_prob

                    if prob < 0.8:
                        one_seq.append(self.seq_mask_token)
                    elif prob < 0.9:
                        one_seq.append(self.rng.randint(1, self.num_items))
                    else:
                        one_seq.append(item)

                    one_seq_labels.append(item)
                else:
                    one_seq.append(item)
                    one_seq_labels.append(0)

            one_seq = one_seq[-self.max_len:]
            one_seq_labels = one_seq_labels[-self.max_len:]

            mask_len = max_len - len(one_seq)

            one_seq = [0] * mask_len + one_seq
            one_seq_labels = [0] * mask_len + one_seq_labels

            seqs_output.append(one_seq)
            labels_output.append(one_seq_labels)

        if self.use_attributes:
            for _map in self.i2attr_map:
                one_map = []
                for seq in seqs_output:
                    one_map.append([_map[item] for item in seq])
                    
                attrs_output.append(one_map)  # 3d. [2d, 2d, ...] / 2d : one attribute
        
        return torch.LongTensor(seqs_output), torch.LongTensor(attrs_output), torch.LongTensor(labels_output)


In [44]:
train_dataset = BertTrainDataset(train, max_len, mask_prob, CLOZE_MASK_TOKEN,
    item_count, rng, i2attr_map, use_attributes)

In [48]:
my_train_collator = MyTrainCollator(rng, CLOZE_MASK_TOKEN, item_count, max_len, use_attributes, i2attr_map)
train_dataloader = data_utils.DataLoader(train_dataset, batch_size=args.train_batch_size,
    shuffle=True, pin_memory=True, collate_fn=my_train_collator)

# valid_dataset

In [58]:
class BertEvalDataset(data_utils.Dataset):
    def __init__(self, args, u2seq, u2answer, max_len, seq_mask_token, negative_samples, i2attr_map, use_attributes):
        self.u2seq = u2seq
        self.users = sorted(self.u2seq.keys())
        self.u2answer = u2answer
        self.max_len = max_len
        self.seq_mask_token = seq_mask_token
        self.negative_samples = negative_samples

        self.i2attr_map = i2attr_map


    def __len__(self):
        return len(self.users)

    def __getitem__(self, index):
        user = self.users[index]
        seq = self.u2seq[user]

        return user, seq
        answer = self.u2answer[user]
        # negs = self.negative_samples[user]
        # attrs = []

        # candidates = answer + negs
        # labels = [1] * len(answer) + [0] * len(negs)
        # seq = self.padding_and_trim(seq, 'seq')

        # if self.use_attributes:
        #     attr_idx = torch.LongTensor([self.i2attr_map[attr_name][item] for item in sequence])
        #     attrs.append(attr_idx)

        # return torch.LongTensor(seq), torch.LongTensor(attrs), torch.LongTensor(candidates), torch.LongTensor(labels)


    def padding_and_trim(self, seq, seq_type):
        if seq_type == 'seq':
            seq = seq + [self.seq_mask_token]
        seq = seq[-self.max_len:]
        padding_len = self.max_len - len(seq)
        seq = [0] * padding_len + seq
        return seq

In [59]:
class MyValidCollator:
    def __init__(self, u2seq, u2answer, max_len, seq_mask_token, negative_samples, use_attributes, i2attr_map):
        self.u2seq = u2seq
        self.u2answer = u2answer
        self.max_len = max_len
        self.seq_mask_token = seq_mask_token
        self.negative_samples = negative_samples
        self.use_attributes = use_attributes
        self.i2attr_map = i2attr_map

    def __call__(self, batch):
        '''
        batch : 1d List of tuples
        [
            (user, [1, 2, 3, .... (batch size)]),
            ...
        ]
        '''
        users_batch = [_[0] for _ in batch]
        seqs_batch = [_[1] for _ in batch]

        seqs_output = []
        candidates_output = []
        labels_output = []

        for user in users_batch:  # users_batch : [user, user, .... (batch size)]
            answer = self.u2answer[user]
            negs = self.negative_samples[user]
            one_seq = self.u2seq[user]

            one_seq_candidates = answer + negs
            one_seq_labels = [1] * len(answer) + [0] * len(negs)
        
            # seq = self.padding_and_trim(seq, 'seq')
            one_seq = one_seq + [self.seq_mask_token]
            one_seq = one_seq[-self.max_len:]
            padding_len = self.max_len - len(one_seq)
            one_seq = [0] * padding_len + one_seq
                
            seqs_output.append(one_seq)
            candidates_output.append(one_seq_candidates)
            labels_output.append(one_seq_labels)

        attrs_output = []
        if self.use_attributes:
            for _map in self.i2attr_map:
                one_map = []
                for seq in seqs_output:
                    one_map.append([_map[item] for item in seq])
                    
                attrs_output.append(one_map)  # attrs_output : 3d [2d, 2d, ...], 2d : one attribute

        return torch.LongTensor(seqs_output), torch.LongTensor(attrs_output), torch.LongTensor(candidates_output), torch.LongTensor(labels_output)

In [60]:
answers='val'
valid_dataset = BertEvalDataset(args, train, answers, max_len,CLOZE_MASK_TOKEN, test_negative_samples, i2attr_map, use_attributes)

In [61]:
my_valid_collator = MyValidCollator(train, val, max_len, CLOZE_MASK_TOKEN, test_negative_samples, use_attributes, i2attr_map)

In [62]:
valid_dataloader = data_utils.DataLoader(valid_dataset, batch_size=args.val_batch_size,
    shuffle=False, pin_memory=True, collate_fn=my_valid_collator)

In [63]:
for _ in valid_dataloader:
    break

In [67]:
printer = PrintInputShape(10)
for t in _:
    printer.print(t, 't')

t : 2d
^	[  0.0000,   0.0000, ..., 1728.0000, 3707.0000]
|	[1107.0000, 1778.0000, ..., 421.0000, 3707.0000]
|	[  0.0000,   0.0000, ..., 3623.0000, 3707.0000]
128			...
|	[  0.0000,   0.0000, ..., 855.0000, 3707.0000]
|	[1121.0000, 1149.0000, ..., 1125.0000, 3707.0000]
v	[  0.0000,   0.0000, ..., 1955.0000, 3707.0000]
	<-- 100 -->

t : 3d
Input's shape : torch.Size([1, 128, 100])
\
 1
  \
  ^	[  0.0000,   0.0000, ...,  75.0000, 302.0000]
  |	[ 19.0000, 254.0000, ..., 275.0000, 302.0000]
  |	[  0.0000,   0.0000, ..., 160.0000, 302.0000]
  128			...
  |	[  0.0000,   0.0000, ...,  33.0000, 302.0000]
  |	[  1.0000,  39.0000, ..., 157.0000, 302.0000]
  v	[  0.0000,   0.0000, ...,  74.0000, 302.0000]
	<-- 100 -->

t : 2d
^	[1440.0000, 3572.0000, ..., 2118.0000, 2227.0000]
|	[1421.0000, 1365.0000, ..., 2856.0000, 455.0000]
|	[102.0000, 1930.0000, ..., 3209.0000, 2235.0000]
128			...
|	[333.0000, 2921.0000, ..., 2259.0000, 2693.0000]
|	[1538.0000, 400.0000, ..., 1748.0000, 1807.0000]
v	[467.000

# use_attributes 설정

In [68]:
use_attributes = True

In [69]:
seq_mask_token = CLOZE_MASK_TOKEN
num_items = item_count
for _ in train_dataloader:
    break

In [70]:
_

(tensor([[1050, 1340, 1565,  ...,  859, 1109,  109],
         [   0,    0,    0,  ..., 3275, 2132, 3278],
         [   0,    0,    0,  ..., 1821, 3395, 3342],
         ...,
         [ 359,  371, 3707,  ..., 2326, 3707, 2328],
         [   0,    0,    0,  ..., 3611, 1818,  925],
         [ 428, 1399,   21,  ...,  940, 3707, 3707]]),
 tensor([[[ 54, 294, 187,  ...,  77, 137, 275],
          [  0,   0,   0,  ..., 160, 150, 269],
          [  0,   0,   0,  ..., 121, 278, 216],
          ...,
          [102,  23, 302,  ..., 254, 302, 254],
          [  0,   0,   0,  ..., 249, 237, 249],
          [160, 160,  61,  ..., 263, 302, 302]]]),
 tensor([[   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         ...,
         [   0,    0, 1274,  ...,    0, 2327,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,  155, 2220]]))

In [72]:
import torch.nn as nn
from model.bert_modules.embedding import TokenEmbedding, PositionalEmbedding

class BERTEmbedding(nn.Module):
    """
    BERT Embedding which is consisted with under features
        1. TokenEmbedding : normal embedding matrix
        2. PositionalEmbedding : adding positional information using sin, cos
        2. SegmentEmbedding : adding sentence segment info, (sent_A:1, sent_B:2)
        sum of all these features are output of BERTEmbedding
    """

    def __init__(self, vocab_size, embed_size, max_len, use_attributes, attrs_vocab_size, i2attr_map, dropout=0.1):
        """
        :param vocab_size: total vocab size
        :param embed_size: embedding size of token embedding
        :param dropout: dropout rate
        """
        super().__init__()
        self.token = TokenEmbedding(vocab_size=vocab_size, embed_size=embed_size)
        self.position = PositionalEmbedding(max_len=max_len, d_model=embed_size)
        self.dropout = nn.Dropout(p=dropout)
        self.embed_size = embed_size
        self.use_attributes = use_attributes
        self.printer = PrintInputShape(5)

        if use_attributes:
            self.i2attr_map = i2attr_map
            self.attrs_emb_mats = []
            for size in attrs_vocab_size:
                self.attrs_emb_mats.append(nn.Embedding(size, embed_size))

    def forward(self, x, attrs):

        self.printer.print(x, f"x")
        self.printer.print(attrs, f"attrs")
        x = self.token(x) + self.position(x) # now x's shape : Batch X seq_len X emb size (3d)
        
        if self.use_attributes:
            attr_sum = torch.zeros_like(x)
            self.printer.print(attr_sum, f"attr_sum before sum")
            for i, (at_idx, one_attr_mat) in enumerate(zip(attrs, self.attrs_emb_mats)):
                self.printer.print(one_attr_mat(at_idx), f"one_attr_mat(at_idx)")
                attr_sum += one_attr_mat(at_idx)
            self.printer.print(attr_sum, f"attr_sum after sum")
            x += attr_sum

        # x's shape : Batch X seq_len (2d)
        return self.dropout(x)

In [73]:
# from model.bert import BERT
from torch import nn as nn

# from model.bert_modules.embedding import BERTEmbedding
from model.bert_modules.transformer import TransformerBlock
from utils import fix_random_seed_as


class BERT(nn.Module):
    def __init__(self, args, i2attr_map):
        super().__init__()
        self.args = args
        fix_random_seed_as(args.model_init_seed)
        # self.init_weights()

        max_len = args.bert_max_len
        num_items = args.num_items
        attrs_each_size = args.attrs_each_size
        attrs_vocab_size = [_ + 2 for _ in attrs_each_size]
        n_layers = args.bert_num_blocks
        heads = args.bert_num_heads
        vocab_size = num_items + 2
        if args.use_attributes:
            use_attributes = args.use_attributes
    
        hidden = args.bert_hidden_units
        self.hidden = hidden
        dropout = args.bert_dropout

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=self.hidden, max_len=max_len, 
        use_attributes=use_attributes, attrs_vocab_size=attrs_vocab_size, i2attr_map=i2attr_map, dropout=dropout)

        # multi-layers transformer blocks, deep network
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(hidden, heads, hidden * 4, dropout) for _ in range(n_layers)])

        self.out = nn.Linear(self.hidden, num_items + 1)

    def forward(self, x, attrs):
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x, attrs)

        # running over multiple transformer blocks
        for transformer in self.transformer_blocks:
            x = transformer.forward(x, mask)

        return self.out(x)

    def init_weights(self):
        pass

In [74]:
model = BERT(args, i2attr_map)

In [76]:
from utils import AverageMeterSet

In [77]:
args.num_epochs

100

# Train

In [78]:
from trainer.utils import recalls_and_ndcgs_for_ks

In [79]:
import torch.optim as optim

In [87]:
average_meter_set = AverageMeterSet()
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
ce = nn.CrossEntropyLoss(ignore_index=0)

for epoch in range(args.num_epochs):
    for batch in train_dataloader:
        batch = [x.to(args.device) for x in batch]
        seqs, attrs, labels = batch
        optimizer.zero_grad()

        logits = model(seqs, attrs)
        logits = logits.view(-1, logits.size(-1))  # (B*T) x V
        labels = labels.view(-1)  # B*T
        loss = ce(logits, labels)
        loss.backward()

        optimizer.step()
        average_meter_set.update('loss', loss.item())
    print(f"Epoch {epoch}, loss {average_meter_set['loss'].avg:.3f}")

    with torch.no_grad():
        for batch_idx, batch in enumerate(valid_dataloader):
            batch = [x.to(args.device) for x in batch]
            seqs, attrs, candidates, labels = batch
            scores = model(seqs, attrs)  # B x T x V
            scores = scores[:, -1, :]  # B x V
            scores = scores.gather(1, candidates)  # B x C

            metrics = recalls_and_ndcgs_for_ks(scores, labels, args.metric_ks)
            for k, v in metrics.items():
                average_meter_set.update(k, v)

Epoch 0, loss 7.809


KeyboardInterrupt: 

In [86]:
average_meter_set.meters['NDCG@10'].avg


0.059370889832886554

In [722]:
min(test_negative_samples.keys())


0

In [723]:
max(test_negative_samples.keys())

6039

In [ ]:
class PrintInputShape:
    def __init__(self, limit=1):
        self.limit = limit
        self.cnt = 1

    # @classmethod
    def print(self, t:np.array, notation):
        if self.cnt <= self.limit:
            print(f"{notation} : {len(t.shape)}d", end='\n')
            if len(t.shape) == 1:
                self.print_1d(t)
                self.print_0d_shape(len(t))
            elif len(t.shape) == 2:
                self.print_2d(t)
            elif len(t.shape) == 3:
                self.print_3d(t)
            else:
                print(f"1d array or over 3d")
            print()
        self.cnt += 1

    def print_0d_shape(self, dim, ws=''):
        print(f"{ws}<-- {dim} -->")

    def print_1d(self, data_1d, ws='', arrow=''):
        if data_1d[0] % int(data_1d[0]):
            sample = f"[{data_1d[0]:8.4f}, {data_1d[1]:8.4f}, {data_1d[2]:8.4f}, ..., {data_1d[-3]:8.4f}, {data_1d[-2]:8.4f}, {data_1d[-1]:8.4f}]"
        else:
            sample = f"[{data_1d[0]:8}, {data_1d[1]:8}, {data_1d[2]:8}, ..., {data_1d[-3]:8}, {data_1d[-2]:8}, {data_1d[-1]:8}]"
        print(f"{ws}{arrow}\t{sample}")
        

    def print_2d(self, t, ws=''):
        t_trim_a = t.clone().detach()[:3]
        t_trim_b = t.clone().detach()[-3:]
        t_trim = np.concatenate([t_trim_a, t_trim_b], axis=0)
        for line, data_1d in enumerate(t_trim, start=1):
            if line == 1:
                self.print_1d(data_1d, ws, arrow='^')
            elif line == 2:
                self.print_1d(data_1d, ws, arrow='|')
            elif line == (len(t_trim)-1):
                self.print_1d(data_1d, ws, arrow='|')
            elif line == len(t_trim):
                self.print_1d(data_1d, ws, arrow='v')
            else:
                self.print_1d(data_1d, ws, arrow='|')

            if line == len(t_trim)//2:  # 중간에 숫자 끼워넣기
                print(f"{ws}{t.size()[0]}\t\t\t...")
        self.print_0d_shape(t.size()[1], ws)

    def print_3d(self, t):
        print(f"Input's shape : {t.size()}")
        ws = ''
        print(f"{ws}\\")
        ws += ' '
        print(f"{ws}{t.size()[0]}")
        ws += ' '
        print(f"{ws}\\")

        self.print_2d(t[-1], ws=ws)

In [ ]:
def test_dataset(use_attributes=False):
    if use_attributes:
        return [1, 2, 3], [4, 5, 6]
    else:
        return [1, 2, 3], []

In [ ]:
def test_forward(x):

    seq, attrs = x

    print(seq)
    print(attrs)

In [ ]:
batch = test_dataset()
batch

([1, 2, 3], [])

In [ ]:
test_forward(batch)

[1, 2, 3]
[]


In [ ]:
a, *c = ( [4, 5, 6], [7, 8, 9])

In [ ]:
a

[4, 5, 6]

In [ ]:
a = {1 : [1, 2,3]}

In [ ]:
b = []

In [ ]:
b.append(a)

In [ ]:
b

[{1: [1, 2, 3]}]

In [ ]:
a = ()

AttributeError: 'tuple' object has no attribute 'append'